#### Setup and load torch

In [ ]:
from IPython.core.display import display, HTML
import sys,cv2
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from Utils.utils import *
from ipywidgets import interact
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

import torch
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

print('Done.')

In [ ]:
%%time

#Load training data
import deepdish as dd

data_folder = 'D:/data/HPA/all/'

d = dd.io.load(data_folder+'poi_0.h5')
X = d['X'].astype(np.float32) / 255.0 # torch likes float images

y = d['labels']
print("Done.")

In [ ]:
#Setup model and optimizer
resnet = models.resnet18(pretrained=False)

resnet.fc = nn.Linear(51200, 28) #adapt last layer to allow larger input images
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet.parameters(),lr=0.0001)

print("Using GPU:",torch.cuda.is_available())
print("Using device ",torch.cuda.get_device_name(0))

print("Done.")

In [ ]:
# Start training

epochs = 1
batch_size = 10

for epoch in range(epochs):
    running_loss = 0
    
    current_buffer_size = X.shape[0]
    for i in range(round(current_buffer_size / batch_size)):
        X_batch = torch.tensor(X[i:i+batch_size].transpose(0,3,1,2))
        y_batch = torch.tensor(y[i:i+batch_size].astype(np.float32),dtype=torch.float)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / ((i+1)*batch_size)))
        
    
    